In [12]:
import mplfinance as mpf
import pandas as pd
from pathlib import Path
import numpy as np
import datetime

In [13]:
# Get BTC OHLCV dataframe
df = pd.read_csv(Path("BTC-USD-20201005-20211005.csv"), index_col="Date", parse_dates=True, infer_datetime_format=True)
display(df.head())
display(df.tail())

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-10-05,10688.034180,10804.000977,10646.443359,10804.000977,10804.000977,1.938519e+10
2020-10-06,10799.778320,10803.456055,10565.197266,10621.664063,10621.664063,2.226496e+10
2020-10-07,10619.803711,10687.268555,10591.963867,10679.136719,10679.136719,1.763439e+10
2020-10-08,10677.625000,10939.799805,10569.823242,10923.627930,10923.627930,2.196212e+10
2020-10-09,NaN,NaN,NaN,NaN,NaN,NaN


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-10-01,43816.742188,48436.011719,43320.023438,48116.941406,48116.941406,4.285064e+10
2021-10-02,48137.468750,48282.062500,47465.496094,47711.488281,47711.488281,3.061435e+10
2021-10-03,47680.027344,49130.691406,47157.289063,48199.953125,48199.953125,2.663812e+10
2021-10-04,48208.906250,49456.777344,47045.003906,49112.902344,49112.902344,3.338317e+10
2021-10-05,49164.750000,51282.441406,49119.136719,51267.457031,51267.457031,3.463356e+10


In [72]:
# Slice of double bottom
intra = df.loc['01-16-2021':'02-17-2021']
intra.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-01-16,36821.648438,37864.367188,35633.554688,36178.140625,36178.140625,5.770619e+10
2021-01-17,36163.648438,36722.351563,34069.320313,35791.277344,35791.277344,5.235985e+10
2021-01-18,35792.238281,37299.285156,34883.843750,36630.074219,36630.074219,4.951170e+10
2021-01-19,36642.234375,37755.890625,36069.804688,36069.804688,36069.804688,5.724420e+10
2021-01-20,36050.113281,36378.328125,33570.476563,35547.750000,35547.750000,6.683457e+10


In [73]:
import plotly.graph_objects as go

In [74]:
# Use plotly to plot candlesticks
fig = go.Figure(data=[go.Candlestick(x=intra.index,
                open=intra['Open'],
                high=intra['High'],
                low=intra['Low'],
                close=intra['Close'])])

fig.show()

In [75]:
intra.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 33 entries, 2021-01-16 to 2021-02-17
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       33 non-null     float64
 1   High       33 non-null     float64
 2   Low        33 non-null     float64
 3   Close      33 non-null     float64
 4   Adj Close  33 non-null     float64
 5   Volume     33 non-null     float64
dtypes: float64(6)
memory usage: 1.8 KB


In [76]:
# Store local max and min ranges
localmin_range = {}
localmax_range = {}

In [77]:
def find_max(cert_candle, curr_candle, prev_candle):
    # get local max
    max_dict = {}
    if curr_candle and prev_candle:       
        if (curr_candle['c'] > cert_candle['c']) & (curr_candle['c'] > prev_candle['c']):
            max_dict = curr_candle
            return max_dict
        else:
            return None
        
def find_min(cert_candle, curr_candle, prev_candle):   
    # get local min
    min_dict = {}
    if curr_candle and prev_candle:
        if (curr_candle['c'] < cert_candle['c']) & (curr_candle['c'] < prev_candle['c']):
            min_dict = curr_candle
            return min_dict
        else:
            return None

In [78]:
# Check if there is a previous bar
def update_cert(cert, index, df):
    # Create certifying candle
    # count number of null values to see if is first row
    null_count = 0
    for value in cert.values():
        if value == None:
            null_count += 1
    
    # update prev dictionary if all null values
    # if first row it will update with those values
    if null_count == 5:
        cert['t'] = index
        cert['o'] = float(df.loc[index]['Open'])
        cert['c'] = float(df.loc[index]['Close'])
        cert['h'] = float(df.loc[index]['High'])
        cert['l'] = float(df.loc[index]['Low'])
        
        # return the certying candle dictionary
        return cert
    
    # update prev with new prev values (new prev is old curr)
    else:
        if index + datetime.timedelta(days=1):
            cert['t'] = (index)
            cert['o'] = float(df.loc[index]['Open'])
            cert['c'] = float(df.loc[index]['Close'])
            cert['h'] = float(df.loc[index]['High'])
            cert['l'] = float(df.loc[index]['Low'])
        
            # return the certying candle dictionary
            return cert
# c[c.iloc[:, 0] == 1]
        
def update_curr(curr, index, df):
    # Create current dictionary of current candle
    idx = df.index  
    
    # format datetime index
    dt_object = (index - datetime.timedelta(days=1)).to_datetime64()
#     print(type(idx))
    
#     print(sum(df.index == dt_object))
    
    if sum(df.index == dt_object) > 0:
        index -= datetime.timedelta(days=1)
        curr['t'] = (index)
        curr['o'] = float(df.loc[index]['Open'])
        curr['c'] = float(df.loc[index]['Close'])
        curr['h'] = float(df.loc[index]['High'])
        curr['l'] = float(df.loc[index]['Low'])
        
        # return the current candle dictionary
        return curr
    else:
        pass
        
def update_prev(prev, index, df):
    # Create the previous candle
    idx = df.index
    

    dt_object = (index - datetime.timedelta(days=2)).to_datetime64()

#     print(sum(df.index == dt_object) )
    if sum(df.index == dt_object) > 0:
        index -= datetime.timedelta(days=2)
        prev['t'] = (index)
        prev['o'] = float(df.loc[index]['Open'])
        prev['c'] = float(df.loc[index]['Close'])
        prev['h'] = float(df.loc[index]['High'])
        prev['l'] = float(df.loc[index]['Low'])
        
        # return the previous candle dictionary
        return prev
    else:
        pass

In [80]:
maxs = []
mins = []

events = []

In [81]:
# Get candles data to find local max's min's
localmax_range = {}
localmin_range = {}
# # store previous candle
# prev = {
#     't' : None,
#     'o' : None,
#     'c' : None,
#     'h' : None,
#     'l' : None   
# }

# # store current candle
# curr = {
#     't' : None,
#     'o' : None,
#     'c' : None,
#     'h' : None,
#     'l' : None 
# }

# store certifying candle (certifies if the max/min is created)
cert = {
    't' : None,
    'o' : None,
    'c' : None,
    'h' : None,
    'l' : None 
}

double_bottom = {'point_a': None, 'point_b': None, 'point_c': None, 'point_d': None, 'point_e': None}

# Init counters
max_counter = 0
min_counter = 0
# key_counter = 0

for index in intra.index:
    cert_candle = update_cert(cert, index, intra)
    curr_candle = update_curr(curr, index, intra)
    prev_candle = update_prev(prev, index, intra)

    
#     maxs.append(find_max(cert_candle, curr_candle, prev_candle))
#     mins.append(find_min(cert_candle, curr_candle, prev_candle))
    localmax_range[max_counter] = find_max(cert_candle, curr_candle, prev_candle)
    localmin_range[min_counter] = find_min(cert_candle, curr_candle, prev_candle)
    
    max_counter += 1
    key_counter += 1
    
    # pont f arbitrary - 5-10 periods out
    # when finished will have 6 values
    event_list = []
    event_list.append(find_max(cert_candle, curr_candle, prev_candle))
    while curr_candle['c'] < event_list[0]['c']:    
         = find_min(cert_candle, curr_candle, prev_candle)
        i
    
    double_bottom['point_c'] = find_max(cert_candle, curr_candle, prev_candle)
    double_bottom['point_d'] = find_min(cert_candle, curr_candle, prev_candle)
    
    print(double_bottom)
    print(f"Third candle: {prev_candle} \nSecond candle: {curr_candle} \nFirst candle: {cert_candle} \n")

# display(localmax_range)
# display(localmin_range

event_list_shannon = []

def create_list_of_event_points_to_add_to_event_dictionar(date_range_dataframe):
    for i in date_range_dataframe check if this date's prices' make up local max
        if yes, store to list labeled event %(+1) = ['a']
        (assign position of a list to a variable)
        check event1_list; if [0] is not null then check if this date's prices' make up a local min and assign value b
        check event1_list; if [1] is not null then check if this date's prices' make up a local max and assign value c; c should be lower than a but higher than b; if conditions are not met, clear event list and start new list iteration on next date
        check event1_list; if [2] is not null then check if this date's prices' make up a local min and assign value d; if this is min then continue to [check for b conditions]  if local min close is in range of b (between b close and b low) then local min event point d or if local min d low is in range of b close and b low then local min is event point d, elif close d is below b low then delete list and start next date     
        check event1_list; if [3] is not null then check if this date's prices'are higher than event point c and assign value e
        check event1_list; if [4] is not null then find price 5 days after point e and label this +5 day price point f
        check event1_list; if [5] is not null then find price 5 days before point a and label this as the"lead up" event point
        

{0: None,
 1: None,
 2: None,
 3: {'t': Timestamp('2021-02-16 00:00:00'),
  'o': 47944.457031,
  'c': 49199.871094,
  'h': 50341.101563,
  'l': 47201.304688},
 4: None,
 5: None,
 6: None,
 7: {'t': Timestamp('2021-02-16 00:00:00'),
  'o': 47944.457031,
  'c': 49199.871094,
  'h': 50341.101563,
  'l': 47201.304688},
 8: None,
 9: None,
 10: None,
 11: {'t': Timestamp('2021-02-16 00:00:00'),
  'o': 47944.457031,
  'c': 49199.871094,
  'h': 50341.101563,
  'l': 47201.304688},
 12: None,
 13: None,
 14: {'t': Timestamp('2021-02-16 00:00:00'),
  'o': 47944.457031,
  'c': 49199.871094,
  'h': 50341.101563,
  'l': 47201.304688},
 15: None,
 16: None,
 17: None,
 18: None,
 19: {'t': Timestamp('2021-02-16 00:00:00'),
  'o': 47944.457031,
  'c': 49199.871094,
  'h': 50341.101563,
  'l': 47201.304688},
 20: None,
 21: None,
 22: {'t': Timestamp('2021-02-16 00:00:00'),
  'o': 47944.457031,
  'c': 49199.871094,
  'h': 50341.101563,
  'l': 47201.304688},
 23: None,
 24: None,
 25: {'t': Timestamp(

{0: None}

In [60]:
# display(maxs)
# display(mins)
# mins_df = pd.DataFrame(mins)
# maxs_df = pd.DataFrame(maxs)

# intra_min_max = pd.concat([intra, maxs_df, mins_df], axis=1)
# display(mins_df)
# display(maxs_df)

In [ ]:
# localmin_range = {}

# prev = {
#     't' : None,
#     'o' : None,
#     'c' : None,
#     'h' : None,
#     'l' : None   
# }

# # store current candle
# curr = {
#     't' : None,
#     'o' : None,
#     'c' : None,
#     'h' : None,
#     'l' : None 
# }

# # store certifying candle (certifies if the max/min is created)
# cert = {
#     't' : None,
#     'o' : None,
#     'c' : None,
#     'h' : None,
#     'l' : None 
# }



# for index in intra.index:
#     cert_candle = update_cert(cert, index, intra)
#     curr_candle = update_curr(curr, index, intra)
#     prev_candle = update_prev(prev, index, intra)
    
    
#     # make max counter
    
    
#     # create dict to store candle information
# #     min_dict = {}
    
#     # to make dictionary keys
    
    
# #     if curr_candle and prev_candle:
# #         # get local min
# #         if (curr_candle['c'] < cert_candle['c']) & (curr_candle['c'] < prev_candle['c']):
# #             min_dict = curr_candle
# #             max_dict['counter'] = max_counter
#     localmin_range[key_counter] = find_min(cert_candle, curr_candle, prev_candle)
    
# #     previous_low = 
#     # store point b
#     double_bottom['b'] = find_min(cert_candle, curr_candle, prev_candle)

#     print(double_bottom)
#     print(f"Third candle: {prev_candle} \nSecond candle: {curr_candle} \nFirst candle: {cert_candle} \n")
# #     df_min = pd.DataFrame(localmin_range).T
# #     df_min.set_index(df_min['t'], inplace=True)
# #             print(df_min)

In [ ]:
# potentially put in previous high as another variable

# b and d must be found to find c, a and e.



# point b is a local min 
# and point c is a local max that is greater than b and less than a
# and point d is less than c and in the range (it's close or wick) of b (close and low wick)
# and e closes above c
# BUY



In [ ]:
# Logic to store double bottom points
# logic to break pattern
# Logic for entry
# Make another column called max and min